In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
import gym as gym
from time import sleep
from time import time

network_t = "cnn"
GAME_NAME="Breakout-v0"
TB_DIR = "saves/breakout/R2_Buf5_1e4/"
env=Atari_Environment(gym.make(GAME_NAME))
state= env.reset()
training_finished = False
LEARNING_RATE = 8e-5
print(env.action_space)

Discrete(4)


/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
def start_method(game_name, save_path=None, restore = False, num_agents = 32, replay_ratio=2, network_type='cnn', learning_rate=8e-5, lrdecay=0.99, MAX_STEPS=1000000, DISCOUNT=0.99, NSTEPS=20):
    if save_path == None:
        save_path = "saves/"+game_name+"_"+replay_ratio+"_Replay"
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        #create Global Net and agents.
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_type,lr=learning_rate,decay=lrdecay)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_type))
            saver = tf.train.Saver(max_to_keep=5)

            #restore parameters?
            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
                rewardlist = np.loadtxt(save_path+"reward.out").tolist()
            else:
                sess.run(tf.global_variables_initializer())
                rewardlist = []
            # create a process for every worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", NSTEPS, DISCOUNT, MAX_STEPS, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR,rewardlist)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), NSTEPS, DISCOUNT, MAX_STEPS, replay_ratio))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # run processes as daemon
            for p in processes:
                p.daemon = True
                p.start()

            # join processes after training.
            coord.join(processes)


In [4]:
tf.reset_default_graph()
start_method(game_name = GAME_NAME, save_path = TB_DIR, restore = True, num_agents=16, replay_ratio=2)

/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from saves/breakout/R2_Buf5_1e4/checkpoints/model589.cptk
5.706417042679972worker_1  using  2 offline steps, per online step

worker_2  using  2 offline steps, per online step
worker_3  using  2 offline steps, per online step
worker_4  using  2 offline steps, per online step
worker_5  using  2 offline steps, per online step
worker_6  using  2 offline steps, per online step
worker_7  using  2worker_8  using  2 offline steps, per online step
 offline steps, per online stepworker_9
  using  2 offline steps, per online step
worker_10  using  2 offline steps, per online step
worker_11  using  2 offline steps, per online step
worker_12  using  2 offline steps, per online step
worker_13  using  2 offline steps, per online step
worker_14  using  2 offline steps, per online step
worker_15  using  2 offline steps, per online step


/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


worker_eval  using  2 offline steps, per online step
Worker  worker_eval At  763  Running/Max:  5.235775338411974 1.0  Frames: 12
pi: [[0.24597816 0.24901009 0.2517717  0.25324014]]
Saving Model


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1322, in _do_call
    return fn(*args)
  File "/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1307, in _run_fn
    options, feed_dict, fetch_list, target_list, run_metadata)
  File "/home/julian/anaconda3/envs/BA/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1409, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Run call was cancelled

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/julian/anaconda3/envs/BA/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/julian/anaconda3/envs/BA/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/ho

KeyboardInterrupt: 